## <font color="white">Zillow Rental Search</font>

##<font color="white">Imports</font>

In [ ]:
from google.colab import output, drive, files # specific to Google Colab
import pandas as pd
import numpy as np
import plotly.express as px
import requests
import warnings
import json
import csv

# settings
warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", None)

## <font color="white">Functions</font>

In [ ]:
def get_listings(api_key, listing_url):
    url = "https://app.scrapeak.com/v1/scrapers/zillow/listing"

    querystring = {
        "api_key": api_key,
        "url":listing_url
    }

    return requests.request("GET", url, params=querystring)

def get_property_detail(api_key, zpid):
    url = "https://app.scrapeak.com/v1/scrapers/zillow/property"

    querystring = {
        "api_key": api_key,
        "zpid":zpid
    }

    return requests.request("GET", url, params=querystring)

def get_zpid(api_key, street, city, state, zip_code=None):
    url = "https://app.scrapeak.com/v1/scrapers/zillow/zpidByAddress"

    querystring = {
        "api_key": api_key,
        "street": street,
        "city": city,
        "state": state,
        "zip_code":zip_code
    }

    return requests.request("GET", url, params=querystring)

## <font color="white">Input API Key</font>

In [ ]:
api_key = "insert_your_api_key"

## <font color="white">Data</font>

### <font color="green">2. Rental Property Listings For Rent</font>

In [ ]:
# zillow search url
rent_listing_url = "insert_your_zillow_search_url"

# get listings
rent_listing_response = get_listings(api_key, rent_listing_url)

In [ ]:
# view all keys
print(rent_listing_response.json().keys())

# check if request is successful
print("Request success:", rent_listing_response.json()["is_success"])

# view count of properies returned in request
num_of_properties = rent_listing_response.json()["data"]["categoryTotals"]["cat1"]["totalResultCount"]
print("Count of properties:", num_of_properties)

dict_keys(['is_success', 'data', 'message'])
Request success: True
Count of properties: 115


In [ ]:
# view all listings
df_rent_listings = pd.json_normalize(rent_listing_response.json()["data"]["cat1"]["searchResults"]["mapResults"])
print("Number of rows:", len(df_rent_listings))
print("Number of columns:", len(df_rent_listings.columns))
df_rent_listings

In [ ]:
##turn DataFrame into csv
print(type(df_rent_listings))

# mount drive
drive.mount("/content/drive", force_remount=False)

# data location
file_path = "/content/drive/My Drive/insert_your_folder/zillow_rental_data.csv"

df_rent_listings.to_csv(file_path, index=False)

<class 'pandas.core.frame.DataFrame'>
Mounted at /content/drive


### <font color="green">3. Print Information for Rental Property Listings</font>

In [ ]:
# view price
px.box(df_rent_listings, x="hdpData.homeInfo.price", title="Rental Price Box Plot")

In [ ]:
# view rent zestimate
px.box(df_rent_listings, x="hdpData.homeInfo.rentZestimate", title="Rent Zestimate Box Plot")

In [ ]:
street = "12315 N Oakleaf Ave"
city = "Tampa"
state = "FL"
# zip_code = "33612" #[optional]

zpid_response = get_zpid(api_key, street, city, state, zip_code=None)

In [ ]:
zpid_response.json()["is_success"]

True

In [ ]:
# view response, top possible zpids
pd.DataFrame(zpid_response.json()["data"])

,zpid,url
0,44905318,https://www.zillow.com/homedetails/12315-N-Oak...
1,44905319,https://www.zillow.com/homedetails/12319-N-Oak...
2,44905317,https://www.zillow.com/homedetails/12311-N-Oak...
3,44905324,https://www.zillow.com/homedetails/12315-Woodl...
4,44905320,https://www.zillow.com/homedetails/1714-Round-...


In [ ]:
print("Zpid:", zpid_response.json()["data"][0]["zpid"])

Zpid: 44905318


# End Notebook